<a href="https://colab.research.google.com/github/MiracleCakra/Machine-Learning_Ganjil_2025/blob/main/Week07_JS07/TUGAS_JS07_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas

Jalankan code berikut pada Google Colab dan PyDroid3 (Android Python) Application di Smartphone Android. Bandingkan hasilnya dan tuliskan analisa anda, tuliskan juga spesifikasi smartphone yang anda gunakan :).

In [2]:
!pip install -q annoy faiss-cpu hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from annoy import AnnoyIndex
import hnswlib
import faiss
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Contoh dataset kecil untuk testing
# -------------------------------
np.random.seed(42)
n_samples = 10000   # jumlah database vector
d = 128             # dimensi
X = np.random.random((n_samples, d)).astype('float32')

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact NN (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=d)
p_hnsw.init_index(max_elements=n_samples, ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, _ = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(d)
index_faiss = faiss.IndexIVFFlat(quantizer, d, 100, faiss.METRIC_L2)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
_, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Tampilkan ringkasan waktu
# -------------------------------
print("\n=== Ringkasan Waktu (detik) ===")
print(f"Exact NN : {time_exact:.3f}")
print(f"Annoy    : {time_annoy:.3f}")
print(f"HNSW     : {time_hnsw:.3f}")
print(f"FAISS    : {time_faiss:.3f}")


Exact NN done in 1.833 s
Annoy done in 0.641 s
HNSW done in 12.922 s
FAISS IVF done in 1.102 s

=== Ringkasan Waktu (detik) ===
Exact NN : 1.833
Annoy    : 0.641
HNSW     : 12.922
FAISS    : 1.102


### Output Pada local (Intel i5 12450HX - 12GB DDR4)
```
Exact NN done in 0.195 s
Annoy done in 0.350 s
HNSW done in 0.442 s
FAISS IVF done in 0.045 s

=== Ringkasan Waktu (detik) ===
Exact NN : 0.175
Annoy    : 0.389
HNSW     : 0.442
FAISS    : 0.045
```

### Output Pada Google Colab free (2xvCPU Intel(R) Xeon(R) CPU @ 2.30GHz, 12.7GB)
```
Exact NN done in 1.833 s
Annoy done in 0.641 s
HNSW done in 12.922 s
FAISS IVF done in 1.102 s

=== Ringkasan Waktu (detik) ===
Exact NN : 1.833
Annoy    : 0.641
HNSW     : 12.922
FAISS    : 1.102
```

## Analisis Perbandingan Kinerja

Berdasarkan perbandingan hasil di laptop lokal (Intel i5 12450HX) dan Google Colab:

*   **Performa Lokal Unggul untuk Komputasi Intensif:** Laptop lokal lebih cepat untuk algoritma seperti FAISS, HNSW, dan Exact NN. Ini karena CPU fisik lokal lebih baik dalam menangani komputasi numerik berat dibandingkan CPU virtual Colab yang bisa ter-*throttle*.
*   **Colab Ideal untuk Eksperimen Ringan:** Google Colab cocok untuk eksperimen dan prototipe awal karena kemudahan aksesnya, meskipun performanya bervariasi.
*   **Annoy Stabil Lintas Platform:** Annoy menunjukkan kinerja yang stabil di Colab, kurang bergantung pada optimasi hardware spesifik.
*   **FAISS Sensitif terhadap Hardware:** Kinerja FAISS sangat dipengaruhi oleh kapabilitas hardware lokal (SIMD, multi-threading), membuatnya sangat cepat di sana.
